<a href="https://colab.research.google.com/github/Anthony5234534/STAT3011-Project-II/blob/main/bg_nbh_gg_Future_3_month_CLV_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:

https://medium.com/@yassirafif/projecting-customer-lifetime-value-using-the-bg-nbd-and-the-gamma-gamma-models-9a937c60fe7f


In [ ]:
!pip install lifetimes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases
from decimal import Decimal
import datetime as dt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00


In [ ]:
# import the orginal data set first
# data set name 'df'

file_name = "https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/OnlineRetail.xlsx"
df = pd.read_excel(file_name, sheet_name='Online Retail')

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.isna().sum()
df.dropna(inplace=True)


In [ ]:
# Dropping rows with negative quantity.
df = df[~df['Quantity'] < 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [ ]:
# Setting data types
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype('str')
df['Description'] = df['Description'].astype('str')
df['StockCode'] = df['StockCode'].astype('str')
df['Country'] = df['Country'].astype('str')
df['UnitPrice'] = df['UnitPrice'].apply(Decimal)

In [ ]:
# Creating the monetary value of the transactions (quantity * price)
df['Monetary'] = df['Quantity'] * df['UnitPrice']
df['Monetary'] = df['Monetary'].apply(Decimal)

In [ ]:
# Computing the summary data (Recency, Frequency, monetary and tenure)
df_rfmt = summary_data_from_transaction_data(transactions = df,
                                         customer_id_col = 'CustomerID',
                                         datetime_col = 'InvoiceDate',
                                         monetary_value_col = 'Monetary')
df_rfmt.head()

,frequency,recency,T,monetary_value
CustomerID,,,,
12346.0,0.0,0.0,325.0,0.000000
12347.0,6.0,365.0,367.0,599.701667
12348.0,3.0,283.0,358.0,301.480000
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000


In [ ]:
# size of the data
diff_time = df['InvoiceDate'].max() - df['InvoiceDate'].min()
diff_time

Timedelta('373 days 04:24:00')

In [ ]:
# Getting the ending date of the calibration period.
end_date_cal = df['InvoiceDate'].min() + dt.timedelta(days=200)
end_date_obs = end_date_cal + (diff_time - dt.timedelta(days=200))

In [ ]:
df_rfmt_cal = calibration_and_holdout_data(transactions=df,
                                          customer_id_col="CustomerID",
                                          datetime_col = "InvoiceDate",
                                          calibration_period_end=end_date_cal,
                                          observation_period_end= end_date_obs)
df_rfmt_cal

,frequency_cal,recency_cal,T_cal,frequency_holdout,duration_holdout
CustomerID,,,,,
12346.0,0.0,0.0,152.0,0.0,173.0
12347.0,3.0,184.0,194.0,3.0,173.0
12348.0,2.0,110.0,185.0,1.0,173.0
12350.0,0.0,0.0,137.0,0.0,173.0
12352.0,3.0,34.0,123.0,3.0,173.0
...,...,...,...,...,...
18273.0,0.0,0.0,84.0,2.0,173.0
18280.0,0.0,0.0,104.0,0.0,173.0
18281.0,0.0,0.0,7.0,0.0,173.0


In [ ]:
l2_coefs = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
l2_list = []
rmse_list = []
for coef in l2_coefs :
    # Fitting the model using the calibration dataset.
    model = BetaGeoFitter(penalizer_coef=coef)
    model.fit(df_rfmt_cal['frequency_cal'],
        df_rfmt_cal['recency_cal'],
        df_rfmt_cal['T_cal'])
    # Predicting the frequency for the holdout period for all customers.
    pred_freq = pd.DataFrame(model.predict(df_rfmt_cal['duration_holdout'],
                                 df_rfmt_cal['frequency_cal'], df_rfmt_cal['recency_cal'], df_rfmt_cal['T_cal']), columns=['pred_frequency']).reset_index()
    # Merging the two dataframes and dropping NaN values.
    new_df = df_rfmt_cal.reset_index().merge(pred_freq, on='CustomerID').dropna()

    # Computing the rmse score
    rmse_score = np.sqrt(mean_squared_error(new_df['frequency_holdout'],new_df['pred_frequency']))
    l2_list.append(coef)
    rmse_list.append(rmse_score)

# Getting the results
resl = pd.DataFrame(np.array(rmse_list), columns=['rmse_score'])\
             .merge(pd.DataFrame(np.array(l2_list), columns=['L2 coefs']), right_index=True, left_index=True)
resl

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(uf

,rmse_score,L2 coefs
0,2.394539,0.3
1,2.386201,0.4
2,2.374557,0.5
3,2.370487,0.6
4,2.364460,0.7
5,2.361638,0.8
6,2.360372,0.9
7,2.366927,1.0


In [ ]:
# Fitting the model using the calibration dataset.
model = BetaGeoFitter(penalizer_coef=0.80)
model.fit(df_rfmt_cal['frequency_cal'],
        df_rfmt_cal['recency_cal'],
        df_rfmt_cal['T_cal'])

<lifetimes.BetaGeoFitter: fitted with 2868 subjects, a: 0.00, alpha: 22.81, b: 0.00, r: 0.29>

In [ ]:
# Predicting the number of purchases in the next 180 days for all customers.
df_rfmt['predicted_purchases'] = model.conditional_expected_number_of_purchases_up_to_time(180,
                                                                                      df_rfmt['frequency'],
                                                                                      df_rfmt['recency'],
                                                                                      df_rfmt['T'])
df_rfmt.dropna(inplace=True)
# Getting rid of negative values.
df_rfmt = df_rfmt[df_rfmt['monetary_value']>0]
df_rfmt

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,frequency,recency,T,monetary_value,predicted_purchases
CustomerID,,,,,
12347.0,6.0,365.0,367.0,599.701667,2.902893
12348.0,3.0,283.0,358.0,301.480000,1.553471
12352.0,6.0,260.0,296.0,368.256667,3.549375
12356.0,2.0,303.0,325.0,269.905000,1.183341
12358.0,1.0,149.0,150.0,683.200000,1.332980
...,...,...,...,...,...
18272.0,5.0,244.0,246.0,487.752000,3.539958
18273.0,2.0,255.0,257.0,76.500000,1.470918
18282.0,1.0,119.0,126.0,77.840000,1.547503


# Gamma Gamma

In [ ]:
# Fitting the GammaGamma model

gg_model = GammaGammaFitter()
gg_model.fit(df_rfmt['frequency'], df_rfmt['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 2790 subjects, p: 2.10, q: 3.45, v: 485.57>

In [ ]:
df_rfmt['pred_monetary'] = gg_model.conditional_expected_average_profit(
        df_rfmt['frequency'],
        df_rfmt['monetary_value'])
df_rfmt

,frequency,recency,T,monetary_value,predicted_purchases,pred_monetary
CustomerID,,,,,,
12347.0,6.0,365.0,367.0,599.701667,2.902893,569.988807
12348.0,3.0,283.0,358.0,301.480000,1.553471,333.762672
12352.0,6.0,260.0,296.0,368.256667,3.549375,376.166864
12356.0,2.0,303.0,325.0,269.905000,1.183341,324.008941
12358.0,1.0,149.0,150.0,683.200000,1.332980,539.930643
...,...,...,...,...,...,...
18272.0,5.0,244.0,246.0,487.752000,3.539958,474.369525
18273.0,2.0,255.0,257.0,76.500000,1.470918,201.781295
18282.0,1.0,119.0,126.0,77.840000,1.547503,260.275833


# CLV prediction
## Result:
predicted_purchases = predicted frequency of future 3 month
predicted_nonetart = predicted average transaction amount of future 3 month

In [ ]:
# Predicting the CLV.
df_rfmt['CLV'] = gg_model.customer_lifetime_value(
    model,
    df_rfmt['frequency'],
    df_rfmt['recency'],
    df_rfmt['T'],
    df_rfmt['monetary_value'],
    time = 3,# In months
    )
df_rfmt

,frequency,recency,T,monetary_value,predicted_purchases,pred_monetary,CLV
CustomerID,,,,,,,
12347.0,6.0,365.0,367.0,599.701667,2.902893,569.988807,811.033735
12348.0,3.0,283.0,358.0,301.480000,1.553471,333.762672,254.145633
12352.0,6.0,260.0,296.0,368.256667,3.549375,376.166864,654.446226
12356.0,2.0,303.0,325.0,269.905000,1.183341,324.008941,187.935338
12358.0,1.0,149.0,150.0,683.200000,1.332980,539.930643,352.779454
...,...,...,...,...,...,...,...
18272.0,5.0,244.0,246.0,487.752000,3.539958,474.369525,823.107427
18273.0,2.0,255.0,257.0,76.500000,1.470918,201.781295,145.482580
18282.0,1.0,119.0,126.0,77.840000,1.547503,260.275833,197.427140
